# Chapter: The theta neuron model
The theta neuron model is defined as follows:
\begin{align*}
\dot{\theta}&=(1-\cos \theta)+(1+\cos \theta) \cdot \iota \hspace{10mm} \theta \in S^1 \\
&=1-\cos \theta + \iota + \iota \cos \theta \\
&=(\iota + 1) + (\iota - 1) \cdot \cos \theta\\
\end{align*}

For $\iota > 0$, $\dot{\theta} > 0$ and $\theta$ moves continuously around the unit circle. Equilibria only exist for $\iota < 0$:
\begin{align*}
\iota + 1 &= (1 - \iota) \cdot \cos \theta\\
\theta^* &= \pm\arccos \left( \frac{\iota + 1}{1-\iota} \right) + 2\pi n\\
\end{align*}


For some of the extreme values of $\frac{\iota + 1}{1-\iota}$, we have that: 

\begin{align*}
\lim_{\iota \rightarrow -1} \theta^* &= \pm \arccos (0) + 2\pi n = 0 + 2\pi n \\
\end{align*}

Concerning the stability:
\begin{align*}
\frac{d}{d\theta}\left((1-\cos \theta)+(1+\cos \theta) \cdot \iota \right) 
&= \sin \theta + \sin \theta \cdot \iota = (1 + \iota)\sin \theta 
= (1 + \iota)\sin\left(\pm\arccos \left( \frac{\iota + 1}{1-\iota} \right) \right) \\
&= \pm (1 + \iota) \cdot \sqrt{1 - \frac{\iota + 1}{1-\iota}}
= \pm (1 + \iota) \cdot \frac{2j\sqrt{\iota}}{1-\iota}
\end{align*}

Where $j = \sqrt{-1}$.

![Testing the model equilibria](Figures/ThetaModelEquilibriumPoints.png)

## QIF
The theta model is closely related to the QIF model:
\begin{align*}
\frac{dV}{dt} = V^2 + \iota
\end{align*}

We can easily transform back and forth between the two different models:
\begin{align*}
V &\equiv \tan\left( \frac{\theta}{2} \right) \\
\theta &\equiv \arctan(2 \cdot V)\\
\frac{dV}{dt} &= \frac{1}{2 \cos^2\left( \frac{\theta}{2} \right)}\frac{d\theta}{dt}
\end{align*}

Using the transformation we obtain:
\begin{align}
\dot{\theta}&=2\left(\cos ^{2}\left(\frac{\theta}{2}\right) \tan ^{2}\left(\frac{\theta}{2}\right)+\cos ^{2}\left(\frac{\theta}{2}\right) \iota \right) \\
&=2\left(\sin ^{2}\left(\frac{\theta}{2}\right)+\cos ^{2}\left(\frac{\theta}{2}\right) \iota \right)
\end{align}

Using $\cos ^{2}\left(\frac{\theta}{2}\right)=\frac{1+\cos \left(\frac{\theta}{2}\right)}{2}$ and $ \sin ^{2}\left(\frac{\theta}{2}\right)=\frac{1-\cos \left(\frac{\theta}{2}\right)}{2}$:
\begin{align*}
\dot{\theta}&=2\left(\frac{1-\cos \theta}{2}+\left(\frac{1+\cos \theta}{2}\right) \iota \right)\\
& = (1-\cos \theta)+(1+\cos \theta) \cdot \iota
\end{align*}

The reverse transformation can be obtained by taking the inverse.

## Frequency response 
See https://grey.colorado.edu/CompCogNeuro/index.php/CCNBook/Neuron/Frequency_Current_Curve and Gutkin2014


*In neuroscience, an F-I curve is the function that relates the net synaptic current flowing into a Neuron to its firing rate (′I′ is the measure for current, 'F' for firing rate.*


To compute the frequency-current response curve we start from the QIF model. Seperate the variables:

\begin{align*}
\frac{dV}{dt} &= V^2 + \iota \\
\frac{dV}{V^2 + \iota} &= dt \\
\end{align*}

Then integrate from the reset potential $-\infty$ at $t=0$ to the threshold potential $+\infty$ at $t = T$, as we want to find T, the time required to go from the reset voltage to the spike threshold. This is tractable for the regime where $\iota > 0$.

\begin{align*}
\int_{-\infty}^{+\infty} \frac{dv}{v^2 + \iota} &= \int_{0}^{T} dt = T \\
\end{align*}

Factoring out $\iota$ we can obtain a recognisable form, and find $T$:

\begin{align*}
T = \int_{-\infty}^{+\infty} \frac{\iota}{\left(\frac{v}{\sqrt{\iota}}\right)^2 + 1} dv \underset{x = \frac{v}{\sqrt{\iota}} \: \:dx = \frac{dv}{\sqrt{\iota}}}{=} \int_{-\infty}^{+\infty} \frac{\iota}{x^2 + 1} dx = \frac{1}{\sqrt{\iota}} \arctan(x) \rvert_{-\infty}^{+\infty} = \frac{1}{\sqrt{\iota}} \left( \frac{\pi}{2} - \left( - \frac{\pi}{2} \right) \right) = \frac{\pi}{\sqrt{\iota}}
\end{align*}

So the frequency of oscillation is proportional to $\sqrt{\iota}$. This result is valid for the theta and QIF models.

## Sensitivity to the input current:
We want to determine how sensitively the period $T$ depends in the input current $\iota$. See 'Introduction To Neuronal Modelling, 2017'.

We define the *condition number*, which measures by how much $T$ changes if $\iota$ is perturbed.
\begin{align*}
\kappa = \left| \frac{dT}{d\iota} \frac{\iota}{T} \right| = \left| \frac{dT / T}{d\iota / \iota}\right|
\end{align*}

*If $\kappa$ = 5 then a 1% change in $\iota$ will change a 5% change in $T$*.

For the theta neuron, use the fact that $\sqrt{\iota} = \frac{\pi}{T}$:
\begin{align*}
\kappa &= \left| -\frac{\pi}{2}\frac{1}{\iota^{3/2}} \frac{\iota}{T} \right| 
= \left| \frac{\pi}{2} \left(\frac{1}{\sqrt{\iota}}\right)^3 \frac{\iota}{T} \right| 
= \left| \frac{\pi}{2} \left(\frac{T}{\pi}\right)^3 \frac{\iota}{T} \right| 
= \frac{1}{2} \left|\left(\frac{T}{\pi}\right)^2 \cdot \left(\frac{\pi}{T}\right)^2 \right| = \frac{1}{2}
\end{align*}

Which means that a 1% increase in the current reduces the period by half.

![Testing the model equilibria](Figures/ThetaNeuronResponseToCurrent.png)

# Chapter: Synaptic Plasticity
Synaptic plasticity is added to the different models by either convoluting in- and output signals of a neuron, or by using an instantanious scheme, or just from the network architecture. There are many possibilitties.


##  'Adaptive Theta Neurons' - Gutkin2014
Add an input to the current that is time-dependant:

\begin{align*}
\dot{\theta}&=(1-\cos \theta)+(1+\cos \theta) \cdot (\iota + \\
\end{align*}